In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\deepak singh\\OneDrive\\Desktop\\comment-toxicity\\End_to_end_comment_toxicity'

In [3]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataLoadingConfig:
	zip_file_path: Path
	destination_dir: Path

In [5]:
from commentToxicity import logger
from commentToxicity.constants import *

In [6]:
from commentToxicity.constants import *
from commentToxicity.utils.common import read_yaml, create_directories

In [7]:
# config/configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_loading_config(self) -> DataLoadingConfig:
        config = self.config.data_loading

        data_loading_config = DataLoadingConfig(
            zip_file_path=config.zip_file_path,
            destination_dir=config.destination_dir
        )

        return data_loading_config


In [23]:
import zipfile
import os
import shutil

class DataLoading:
    def __init__(self, config: DataLoadingConfig):
        self.config = config

    def data_unzip(self):
        def unzip_file(zip_path, extract_to):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
                # Check for nested zip files
                for member in zip_ref.namelist():
                    # If the member is a zip file, unzip it recursively
                    if member.endswith('.zip'):
                        nested_zip_path = os.path.join(extract_to, member)
                        unzip_file(nested_zip_path, extract_to)

        try:
            extraction_dir = os.path.dirname(self.config.zip_file_path)
            unzip_file(self.config.zip_file_path, extraction_dir)

            logger.info(f"Data extracted in {extraction_dir}")

        except Exception as e:
            raise e

    def load_data(self):
        try:
            root_dir = os.path.dirname(self.config.zip_file_path)
            train_file_path = os.path.join(root_dir, "train.csv")

            shutil.copy(train_file_path, self.config.destination_dir)

            logger.info(f"Data copied to {self.config.destination_dir}")
        except Exception as e:
            raise e


In [24]:
# pipeline

# from cnnClassifier.config.configuration import ConfigurationManager
# from cnnClassifier.components.data_ingestion import DataIngestion
# from cnnClassifier import logger



STAGE_NAME = "Data Loading stage"

class DataLoadingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_loading_config = config.get_data_loading_config()
        data_loading = DataLoading(config=data_loading_config)
        data_loading.data_unzip()
        data_loading.load_data()


if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataLoadingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e

[2024-09-25 23:10:14,595: INFO: 2252534611: >>>>>> stage Data Loading stage started <<<<<<]
[2024-09-25 23:10:14,598: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-25 23:10:14,599: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-25 23:10:15,804: INFO: 2001161588: Data extracted in data/raw]
[2024-09-25 23:10:16,102: INFO: 2001161588: Data copied to data/interim/]
[2024-09-25 23:10:16,104: INFO: 2252534611: >>>>>> stage Data Loading stage completed <<<<<<

x==========x]
